In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
from PIL import Image
import io
import shutil
import glob
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.io import imread
from skimage.transform import resize
from skimage import color
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing import image
import cv2



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# zip_path = '/kaggle/img-data/images.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # Распаковка ZIP-архива
    zip_file.extractall()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/img-data/sample_submission.csv
/kaggle/input/img-data/train.csv
/kaggle/input/img-data/test.csv
/kaggle/input/img-data/images/images/YC74CENUBg.png
/kaggle/input/img-data/images/images/nCnUACoyVb.jpg
/kaggle/input/img-data/images/images/TevFHDsqVC.jpg
/kaggle/input/img-data/images/images/NYOMKoX0pw.png
/kaggle/input/img-data/images/images/HmlE9CeaPQ.jpeg
/kaggle/input/img-data/images/images/LxWQuoeNH5.jpeg
/kaggle/input/img-data/images/images/Aq3vdt4cIe.jpg
/kaggle/input/img-data/images/images/PVIm8Nqmi3.jpg
/kaggle/input/img-data/images/images/9G5hsaq7xp.jpeg
/kaggle/input/img-data/images/images/QjSAjqK0JF.jpeg
/kaggle/input/img-data/images/images/ZTVQ9l5B7E.jpeg
/kaggle/input/img-data/images/images/ZzMdhgsWr6.jpg
/kaggle/input/img-data/images/images/f9TyKFbNKI.jpg
/kaggle/input/img-data/images/images/w1FO6Q72e6.jpeg
/kaggle/input/img-data/images/images/ilN2I566Wc.png
/kaggle/input/img-data/images/images/dF53CJ3grq.jpg
/kaggle/input/img-data/images/images/9hEvvkphC0.jpeg

NameError: name 'zip_path' is not defined

In [17]:
train = pd.read_csv('/kaggle/input/img-data/train.csv')
test = pd.read_csv('/kaggle/input/img-data/test.csv')
sumbmiss = pd.read_csv('/kaggle/input/img-data/sample_submission.csv')

In [18]:
image_folder = '/kaggle/input/img-data/images/images/'
image_files = os.listdir(image_folder)

In [19]:
# Получаем список файлов в папке
image_files = os.listdir(image_folder)

# Извлекаем имена файлов без расширения
image_names = [os.path.splitext(file)[0] for file in image_files]

In [20]:
# создаем папки для изображений с метками 0 и 1
if not os.path.exists('AI'):
    os.makedirs('AI')
if not os.path.exists('photo'):
    os.makedirs('photo')
if not os.path.exists('Test'):
    os.makedirs('Test')

In [21]:
train['id'] = train['id'].str.replace('.jpg', '').str.replace('.png', '').str.replace('.jpeg', '')
train['target'] = train['target'].astype('int8')

In [22]:
test['id'] = test['id'].str.replace('.jpg', '').str.replace('.png', '').str.replace('.jpeg', '').str.replace('.webp', '')

In [23]:
# Проходим по всем строкам датафрейма и сортируем тренировочный набор изображений
for index, row in train.iterrows():
    filename = row['id']
    target = row['target']

# формируем полный путь к изображению
# проверяем, существует ли изображение

    if filename in image_names:
        image_files = glob.glob(os.path.join(image_folder, filename + '.*'))
        if image_files:
            path_to_image = image_files[0]  # Берем первый файл из списка
    
            if target == 0:
                shutil.copy(path_to_image, 'photo')
            elif target == 1:
                shutil.copy(path_to_image, 'AI')

In [24]:
image_sizes = {}

# Перебор всех файлов в папке
for filename in os.listdir(image_folder):
    # Путь к файлу
    file_path = os.path.join(image_folder, filename)
    # Проверка, является ли файл изображением
    if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif', '.webp')):
        # Открытие изображения
        img = Image.open(file_path)
        # Получение размера изображения
        width, height = img.size
        # Добавление размера изображения в словарь
        if (width, height) in image_sizes:
            image_sizes[(width, height)] += 1
        else:
            image_sizes[(width, height)] = 1

# Вывод размеров изображений и их количества

for size, count in image_sizes.items():
    print(f'{size}: {count}')
#     if size[1] < 330:
#         print(f'{size}: {count}')

(738, 684): 1
(375, 500): 86
(333, 500): 52
(746, 698): 2
(500, 325): 2
(500, 333): 175
(500, 334): 34
(500, 301): 1
(500, 364): 1
(1024, 1024): 252
(500, 375): 270
(500, 344): 1
(724, 742): 6
(500, 353): 2
(700, 760): 1
(500, 445): 1
(500, 400): 9
(500, 351): 4
(500, 350): 2
(500, 332): 33
(500, 358): 6
(710, 730): 2
(746, 748): 4
(668, 669): 1
(740, 724): 1
(500, 372): 4
(746, 750): 2
(357, 500): 3
(400, 317): 1
(477, 500): 1
(608, 750): 1
(334, 500): 12
(500, 356): 2
(287, 500): 1
(742, 688): 1
(337, 500): 1
(500, 396): 1
(744, 738): 15
(708, 748): 1
(500, 287): 1
(500, 338): 2
(690, 730): 1
(500, 393): 1
(400, 500): 6
(430, 436): 1
(732, 730): 2
(390, 500): 2
(500, 308): 1
(738, 734): 5
(500, 319): 3
(732, 748): 2
(399, 500): 2
(742, 748): 1
(738, 470): 1
(750, 738): 2
(500, 470): 1
(500, 260): 1
(401, 500): 4
(950, 806): 1
(500, 270): 1
(355, 500): 1
(500, 329): 1
(500, 500): 12
(332, 500): 7
(722, 730): 2
(500, 324): 3
(740, 752): 6
(738, 742): 5
(500, 330): 4
(512, 512): 9
(726,

In [25]:
# Проходим по всем строкам датафрейма и сортируем тестовый набор изображений
test_data = []
for row in test['id']:
    filename = row

# формируем полный путь к изображению
# проверяем, существует ли изображение

    if filename in image_names:
        image_files = glob.glob(os.path.join(image_folder, filename + '.*'))
        if image_files:
            path_to_image = image_files[0]  # Берем первый файл из списка
            shutil.copy(path_to_image, 'Test')
            test_data.append(path_to_image)
    else: print(row)

In [35]:
data = []
labels = []

In [ ]:
# from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
# from keras.applications.vgg16 import preprocess_input

In [ ]:
# # Загрузка предварительно обученной модели VGG16 без верхних слоев
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [36]:
data_dir = '/kaggle/working/'
categoryes = ['photo', 'AI']

# size = 500

for index, category in enumerate(categoryes):
    for file in os.listdir(os.path.join(data_dir, category)):
        img_path = os.path.join(data_dir, category, file)
        
        image = cv2.imread(img_path)
        
        # Проверяем размер изображения
        width, height, channels = image.shape
        if width < height :
            # Переворачиваем изображение на 90 градусов по часовой стрелке
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
            # Сохраняем измененное изображение
            cv2.imwrite(file, image) 

        # Проверяем количество цветовых каналов
        if channels > 3:
            # Изменяем количество каналов на 3
#             image = image[:,:,:3]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Изменяем размер 
        image = cv2.resize(image, (500, 500))

        # Извлекаем признаки с помощью SIFT
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(image, None)
        
        # Преобразуем описатели в один большой массив
        descriptors_array = descriptors_array.reshape(-1)

        
        
        data.append(descriptors_array.flatten())
        labels.append(index)
        

        
        ################################################
        #### ЭКСПЕРЕМЕНТЫ С ПОВОРОТАМИ ИЗОБРАЖЕНИЙ #####
#         width, height, channels = features.shape
#         if width < 400:
#             image = cv2.rotate(features, cv2.ROTATE_90_CLOCKWISE)
    
#         image = resize(image, (500, 400))
        
#         # Сохраняем измененное изображение
#         cv2.imwrite(file, image)   
        ################################################
        
        
        
        # При использовании модели VGG16
#         img_array = np.array(image)
#         expanded_img_array = np.expand_dims(img_array, axis=0)
#         preprocessing_img = preprocess_input(expanded_img_array)
#         feature = base_model.predict(preprocessing_img).flatten()
        

#         data.append(feature)
#         labels.append(index)

In [37]:
# Определяем максимальную длину строки в списке data
max_length = max([len(row) for row in data])

# Заполняем нулями до одинаковой длины всех строк
data = [np.pad(row, (0, max_length - len(row)), 'constant') for row in data]

In [28]:
image_sizes_new = {}
for index, category in enumerate(categoryes):
    for file in os.listdir(os.path.join(data_dir, category)):
        img_path = os.path.join(data_dir, category, file)
        
        image = cv2.imread(img_path)
        # Проверка, является ли файл изображением
        #if os.path.isfile(img_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif', '.webp')):
        # Открытие изображения
        img = Image.open(img_path)
        # Получение размера изображения
        width, height = img.size
        # Добавление размера изображения в словарь
        if (width, height) in image_sizes_new:
            image_sizes_new[(width, height)] += 1
        else:
            image_sizes_new[(width, height)] = 1

# Вывод размеров изображений и их количества

for size, count in image_sizes_new.items():
    print(f'{size}: {count}')

(375, 500): 57
(333, 500): 35
(500, 336): 3
(500, 375): 188
(500, 332): 24
(400, 500): 3
(500, 319): 2
(500, 334): 22
(500, 400): 5
(424, 500): 2
(355, 500): 1
(399, 500): 1
(500, 324): 2
(326, 500): 2
(500, 333): 113
(500, 320): 1
(500, 358): 5
(500, 295): 1
(500, 351): 4
(500, 500): 7
(500, 337): 3
(500, 309): 2
(500, 494): 1
(406, 500): 1
(500, 349): 2
(335, 500): 6
(500, 294): 1
(334, 500): 10
(327, 500): 1
(390, 500): 2
(363, 500): 1
(500, 341): 2
(500, 281): 2
(500, 388): 1
(500, 331): 3
(500, 222): 1
(500, 376): 1
(219, 500): 1
(500, 335): 7
(500, 342): 2
(332, 500): 6
(500, 395): 1
(450, 338): 2
(350, 284): 1
(365, 500): 2
(297, 500): 1
(500, 323): 1
(500, 377): 2
(500, 372): 2
(500, 287): 1
(500, 470): 1
(500, 403): 2
(315, 500): 1
(500, 272): 1
(342, 500): 1
(357, 500): 2
(500, 430): 1
(500, 446): 1
(500, 326): 5
(414, 500): 1
(413, 500): 2
(500, 344): 1
(500, 382): 1
(380, 500): 1
(352, 500): 1
(500, 253): 1
(330, 500): 2
(500, 374): 5
(500, 338): 2
(500, 343): 1
(408, 500):

In [38]:
len(data)

1012

In [39]:
for i in data:
    print(len(i))

535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424
535424

In [40]:
len(labels)

1012

In [ ]:
# data_dir = '/kaggle/working/'
# categoryes = ['photo', 'AI']

# # size = 500

# for index, category in enumerate(categoryes):
#     for file in os.listdir(os.path.join(data_dir, category)):       
#         img_path = os.path.join(data_dir, category, file)
        
#         img = imread(img_path)
        
#         height, width, _ = img.shape
        
#         # Находим координаты для квадрата размером 330x330
#         start_x = (width - 330) // 2
#         start_y = (height - 330) // 2
        
#         # Извлекаем квадрат из центра
#         cropped_image = img[start_y:start_y + 330, start_x:start_x + 330]
            
#         # Преобразуем изображение в оттенки серого, если оно цветное
#         if len(img.shape) == 3 and img.shape[2] > 3:
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         else:
#             gray = img
                
#         # Создаем объект ORB
#         orb = cv2.ORB_create()

#         # Находим ключевые точки и описатели
#         keypoints, descriptors = orb.detectAndCompute(gray, None)

#         # Преобразуем описатели в один большой массив
#         descriptors_array = descriptors.reshape(-1)
        
        
#         data.append(descriptors_array.flatten())
#         labels.append(index)

In [ ]:
# # Определяем максимальную длину описателей
# max_descriptor_length = max([len(descriptors_array) for descriptors_array in data])

# # Добавляем пустые описатели к более коротким описателям
# for i in range(len(data)):
#     descriptors_array = data[i]
#     if len(descriptors_array) < max_descriptor_length:
#         padding = max_descriptor_length - len(descriptors_array)
#         descriptors_array = np.pad(descriptors_array, (0, padding), 'constant')
#         data[i] = descriptors_array

# # Преобразуем список описателей в массив NumPy
# arr_data = np.array(data)

In [ ]:
# data_dir = '/kaggle/working/'
# categoryes = ['photo', 'AI']

# size = 500

# for index, category in enumerate(categoryes):
#     for file in os.listdir(os.path.join(data_dir, category)):       
#         img_path = os.path.join(data_dir, category, file)
        
#         img = imread(img_path)
#         # т.к. есть изображения с более чем 3 канала, то преобразуем их в RGB
#         if len(img.shape) != 3 or img.shape[2] != 3:
#             img = img[:, :, :3]

#         # Проверка корректности форматов:
#         if len(img.shape) != 3 or img.shape[2] != 3:
#             print("Image has wrong format:", img_path, img.shape)

#         img = resize(img, (size, size))
        
#         data.append(img.flatten())
#         labels.append(index)

In [ ]:
# for i in data:
#     print(len(i))

In [ ]:
# arr_data


In [ ]:
# data

In [41]:
data = [x / 255.0 for x in data]

In [ ]:
# arr_data = [x / 255.0 for x in arr_data]

In [ ]:
# data_img = np.array(arr_data)
# labels_img = np.array(labels_img)

In [42]:
data_img = np.array(data)
labels_img = np.array(labels)

In [43]:
data_img.shape

(1012, 535424)

In [ ]:
# Для метода опорных векторов
data_img = data_img.reshape(data_img.shape[0], -1)

In [ ]:
data_img.shape

In [ ]:
# data_dir = '/kaggle/working/'
# categoryes = ['photo', 'AI']

# for index, category in enumerate(categoryes):
#     for file in os.listdir(os.path.join(data_dir, category)):       
#         img_path = os.path.join(data_dir, category, file)
        
#         img = imread(img_path)
#         # т.к. есть изображения с более чем 3 канала, то преобразуем их в RGB
#         if len(img.shape) != 3 or img.shape[2] != 3:
#             img = img[:, :, :3]

#         # Проверка корректности форматов:
#         if len(img.shape) != 3 or img.shape[2] != 3:
#             print("Image has wrong format:", img_path, img.shape)

#         img = resize(img, (650, 650))
        
#         data.append(img.flatten())
#         labels.append(index)

In [44]:
x_train, x_test, y_train, y_test = train_test_split(data_img, labels_img, test_size=0.2, random_state=42)

In [50]:
from sklearn import svm

In [51]:
# Обучаем модель
clf = svm.SVC(kernel='linear', C=0.1, gamma=0.01, probability=True)
clf.fit(x_train, y_train)

SVC(C=0.1, gamma=0.01, kernel='linear', probability=True)

In [ ]:
# clf_param_grid = {'C': [0.1, 0.5, 1, 10, 100],
#               'gamma': [0.001, 0.01, 0.1, 1]}

In [ ]:
# clf_grid_search = GridSearchCV(clf, clf_param_grid, cv=5, scoring='accuracy')
# clf_grid_search.fit(x_train, y_train)

In [ ]:
# best_params = clf_grid_search.best_params_

In [ ]:
# best_params

In [ ]:
# clf_best = svm.SVC(C=best_params['C'], gamma=best_params['gamma'], probability=True)
# clf_best.fit(x_train, y_train)

In [53]:
# Прогнозируем классы для тестовой выборки
y_pred = clf.predict(x_test)

In [ ]:
# # Прогнозируем классы для тестовой выборки
# y_pred = clf_best.predict(x_test)

In [54]:
# Вычисляем точность
accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

Точность: 0.6551724137931034


In [ ]:
data_dir = '/kaggle/working/Test'
test_data = []
for file in os.listdir(data_dir):       
    img_path = os.path.join(data_dir, file)
    
    img = imread(img_path)
        
    height, width, _ = img.shape
        
    # Находим координаты для квадрата размером 330x330
    start_x = (width - 330) // 2
    start_y = (height - 330) // 2
        
    # Извлекаем квадрат из центра
    cropped_image = img[start_y:start_y + 330, start_x:start_x + 330]
            
    # Преобразуем изображение в оттенки серого, если оно цветное
    if len(img.shape) == 3 and img.shape[2] > 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
                
    # Создаем объект ORB
    orb = cv2.ORB_create()

    # Находим ключевые точки и описатели
    test_keypoints, test_descriptors = orb.detectAndCompute(gray, None)

    # Преобразуем описатели в один большой массив
    test_descriptors_array = test_descriptors.reshape(-1)
        
    test_data.append(test_descriptors_array.flatten())
    

In [ ]:
# Определяем максимальную длину описателей
max_descriptor_length = max([len(test_descriptors_array) for test_descriptors_array in test_data])

# Добавляем пустые описатели к более коротким описателям
for i in range(len(test_data)):
    test_descriptors_array = test_data[i]
    if len(test_descriptors_array) < max_descriptor_length:
        padding = max_descriptor_length - len(test_descriptors_array)
        test_descriptors_array = np.pad(test_descriptors_array, (0, padding), 'constant')
        test_data[i] = test_descriptors_array

# Преобразуем список описателей в массив NumPy
test_arr_data = np.array(test_data)

In [ ]:
test_arr_data

In [ ]:
test_arr_data = [x / 255.0 for x in test_arr_data]

In [ ]:
test_data_img = test_arr_data

In [ ]:
test_data_img = np.array(test_data_img)

In [ ]:
# Для метода опорных векторов
test_data_img = test_data_img.reshape(test_data_img.shape[0], -1)

In [ ]:
test_data_img.shape

In [ ]:
# Прогнозируем классы для окончательной тестовой выборки
final_y = prod_final.predict(test_data_img)

In [ ]:
final_y

In [ ]:
probabilities = prod_final.predict_proba(test_data_img)

In [ ]:
# Вероятность класса 1
probability_class_1 = probabilities[:, 1]

In [ ]:
len(probability_class_1)

In [ ]:
probability_class_1

In [ ]:
sumbmiss['target'] = probability_class_1

In [ ]:
sumbmiss

In [ ]:
sumbmiss.to_csv('submission.csv', index=False)